<a href="https://colab.research.google.com/github/Neafiol/Tinkoff/blob/master/Rnn/Rnn_sentiment_dz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

# Сентимент анализ 

пишем сами с нуля

<img src="https://github.com/bentrevett/pytorch-sentiment-analysis/raw/bf8cc46e4823ebf9af721b595501ad6231c73632/assets/sentiment1.png">

In [0]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

In [3]:
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL, root="./data")

aclImdb_v1.tar.gz:   0%|          | 131k/84.1M [00:00<01:11, 1.18MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 60.9MB/s]


In [4]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

print(vars(train_data.examples[0])['text'])
len(train_data.examples)

Number of training examples: 25000
Number of testing examples: 25000
['along', 'with', 'it', "'s", 'partner', ',', 'this', 'is', 'the', 'greatest', 'piece', 'of', 'animation', 'ever', 'created', '.', 'the', 'images', 'and', 'styles', 'are', 'amazing', ',', 'and', 'match', 'perfectly', 'with', 'the', 'story', 'which', 'is', 'a', 'brilliantly', 'realistic', 'reinterpretation', 'of', 'our', 'own', 'world', ',', 'where', 'is', 'has', 'been', ',', 'and', 'where', 'it', 'could', 'go', '.', 'quite', 'affecting', 'and', 'sometimes', 'painful', 'to', 'watch', ',', 'it', 'it', 'a', 'masterpiece', 'of', 'the', 'visual', 'art', '.']


25000

In [5]:
# Сделаем еще eval
import random

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

# Сделаем словарь
TEXT.build_vocab(train_data, max_size=25000)
LABEL.build_vocab(train_data)

print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

print(TEXT.vocab.itos[:10])
vars(LABEL.vocab)

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2
['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


{'freqs': Counter({'neg': 8810, 'pos': 8690}),
 'itos': ['neg', 'pos'],
 'stoi': defaultdict(<function torchtext.vocab._default_unk_index>,
             {'neg': 0, 'pos': 1}),
 'vectors': None}

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# собираем батчи так, чтобы в каждом батче были примеры наиболее похожей длины
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    device=device)

## Делаем модель

<img src="https://github.com/bentrevett/pytorch-sentiment-analysis/raw/bf8cc46e4823ebf9af721b595501ad6231c73632/assets/sentiment7.png" width="450">

* В эмбеддер (emb = [torch.nn.Embedding(num_embeddings, embedding_dim)](https://pytorch.org/docs/stable/nn.html?highlight=embedding#torch.nn.Embedding)) запихиваем тензор размерностью **[sentence length, batch size]**
* Эмбеддер возвращает тензор размерностью **[sentence length, batch size, embedding dim]**
* RNN (torch.nn.RNN(embedding_dim, hidden_dim)) возвращает 2 тензора, *output* размера [sentence length, batch size, hidden dim] и *hidden* размера [1, batch size, hidden dim]

In [0]:
class RNN(nn.Module):
    def __init__(self, batch_size,input_dim, embedding_dim, hidden_dim, output_dim,dropout=0.3):
        super().__init__()
        
        
        self.batch_size = batch_size

        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.basic_rnn = nn.LSTM(self.embedding_dim, self.hidden_dim) 
        self.clas = nn.Linear(self.hidden_dim, self.output_dim)
        
        self.sm = nn.Softmax(dim = 2)
        self.dropout = nn.Dropout(dropout) 
        
        
    def init_hidden(self):
        # (num_layers, batch_size, n_neurons)
        return (torch.zeros(1, self.batch_size, self.hidden_dim))
      
    def forward (self, text):
      

        #text = [sent len, batch size]
        
        h=self.init_hidden()
#         print('text: ', text.size())
#         print('text: ', text)
        embedded = self.embedding (text)
        
        out, (hidden, c_0) = self.basic_rnn(embedded) 
        
#         print("hidden: ", hidden.size())
#         print("out: ", out.size())
        
        output=self.dropout(hidden)
        output=self.clas(output)

        
  
#         print(output.size())
        
        
        output=self.sm(output)


        return output[0].view(-1,self.output_dim)

In [0]:
input_dim = len(TEXT.vocab.freqs)

batch_size=64
embedding_dim = 12
hidden_dim = 32
output_dim = 2
N_EPHOCS = 100

In [61]:
input_dim

102341

In [0]:
import torch.optim as optim

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Model instance
model = RNN(batch_size,input_dim, embedding_dim, hidden_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()
  

In [0]:
for epoch in range(N_EPHOCS):  # loop over the dataset multiple times
    train_running_loss = 0.0
    train_acc = 0.0
    model.train()
    
    # TRAINING ROUND
    for i, data in enumerate(train_iterator):
         # zero the parameter gradients
        optimizer.zero_grad()
        
        # get the inputs
        inputs, labels = data.text, data.label.long()
        
#         print("inputs",inputs.size())
        
        if(inputs.shape[1]!=batch_size):
          continue

        # forward + backward + optimize
        
        out = model(inputs)
        

        loss = criterion(out, labels)
        
        loss.backward()
        optimizer.step()

        train_running_loss += loss.detach().item()
        train_acc += get_accuracy(out, labels, BATCH_SIZE)
#         break
#     break
         
    model.eval()
    print('Epoch:  %d | Loss: %.4f | Train Accuracy: %.2f | loss: %.2f' 
          %(epoch, train_running_loss / i, train_acc/i,loss))

Epoch:  0 | Loss: 0.6938 | Train Accuracy: 49.23 | loss: 0.69
Epoch:  1 | Loss: 0.6933 | Train Accuracy: 49.55 | loss: 0.69
Epoch:  2 | Loss: 0.6931 | Train Accuracy: 49.93 | loss: 0.69
Epoch:  3 | Loss: 0.6955 | Train Accuracy: 49.67 | loss: 0.70
Epoch:  4 | Loss: 0.6957 | Train Accuracy: 49.85 | loss: 0.68
Epoch:  5 | Loss: 0.6946 | Train Accuracy: 50.10 | loss: 0.70
Epoch:  6 | Loss: 0.6935 | Train Accuracy: 49.39 | loss: 0.69
Epoch:  7 | Loss: 0.6922 | Train Accuracy: 50.27 | loss: 0.69


In [0]:
help(get_accuracy)

In [43]:
inputs.shape

torch.Size([64])